In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from typing import Tuple, Optional, Dict, Any, List

# Local imports
import sys
sys.path.append("../../..")
from src.colors import COLORS

# Configure matplotlib
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white'
})

print("Imports successful!")

---
## Section 1: Introduction — From Matrices to Networks

In the previous notebooks, we learned to compute **connectivity matrices** — square matrices where each entry represents the coupling strength between two brain regions or channels. These matrices contain rich information about brain network organization, but they can be overwhelming: a 64-channel EEG yields a 64×64 matrix with over 2,000 unique connections!

How do we **summarize** such complex data? How do we identify the most important regions? How do we compare network organization across conditions or groups?

**Graph theory** provides the answer. It's a mathematical framework for analyzing networks — and connectivity matrices are networks in disguise! A connectivity matrix is simply a different way of representing a graph:

- **Nodes** (vertices) = EEG channels or brain regions
- **Edges** (connections) = significant connectivity values
- **Edge weights** = connectivity strength

Graph theory offers a rich toolkit of **metrics** that summarize network properties:

- **Node metrics**: How important is each brain region? (hub identification)
- **Local metrics**: Are neighboring regions clustered together? (functional modules)
- **Global metrics**: How efficiently does information flow through the network?
- **Topology**: Is the network random, regular, or "small-world"?

For **hyperscanning**, graph theory becomes even more powerful. We can create a **dual-brain network** combining channels from both participants. This unified graph reveals not just within-brain organization, but also how the two brains connect to each other — with "social hub" regions that bridge the inter-brain gap.

> 💡 **Key insight**: Graph theory transforms connectivity matrices into interpretable network summaries. Instead of examining thousands of connection values, we extract a handful of meaningful metrics that characterize network organization.

---
## Section 2: Graphs — Nodes and Edges

A **graph** $G = (V, E)$ consists of:

- **V** = set of **vertices** (nodes)
- **E** = set of **edges** (connections between nodes)

For EEG connectivity analysis:
- Nodes = electrodes or channels
- Edges = significant connectivity between channel pairs

### The Adjacency Matrix

A graph can be represented as an **adjacency matrix** $A$:

$$A_{ij} = \begin{cases} 1 & \text{if edge exists between } i \text{ and } j \\ 0 & \text{otherwise} \end{cases}$$

This should look familiar — our connectivity matrices ARE adjacency matrices (with weights)!

### Types of Graphs

| Type | Description | Adjacency Property |
|------|-------------|--------------------|
| **Undirected** | Edges have no direction | $A$ is symmetric: $A_{ij} = A_{ji}$ |
| **Directed** | Edges have direction | $A$ may be asymmetric |
| **Binary** | Edges present (1) or absent (0) | $A_{ij} \in \{0, 1\}$ |
| **Weighted** | Edges have continuous strengths | $A_{ij} \in \mathbb{R}$ |

Most connectivity metrics (PLV, coherence, correlation) produce **weighted undirected** graphs.

Transfer entropy produces **weighted directed** graphs.

Let's visualize these concepts!

In [ ]:
# Visualization 1: Simple graph and its adjacency matrix

def draw_node(
    ax: plt.Axes,
    x: float,
    y: float,
    label: str,
    color: str,
    size: float = 0.15
) -> None:
    """Draw a node (circle with label)."""
    circle = plt.Circle((x, y), size, color=color, ec='white', lw=2, zorder=3)
    ax.add_patch(circle)
    ax.text(x, y, label, ha='center', va='center', fontsize=12, 
            fontweight='bold', color='white', zorder=4)


def draw_edge(
    ax: plt.Axes,
    x1: float,
    y1: float,
    x2: float,
    y2: float,
    color: str = COLORS["grid"],
    width: float = 2
) -> None:
    """Draw an edge (line between nodes)."""
    ax.plot([x1, x2], [y1, y2], color=color, lw=width, zorder=1)


# Create a simple 6-node graph
n_nodes = 6
node_labels = ["A", "B", "C", "D", "E", "F"]

# Positions in a circle
angles = np.linspace(0, 2 * np.pi, n_nodes, endpoint=False) - np.pi / 2
radius = 1.0
positions = np.column_stack([radius * np.cos(angles), radius * np.sin(angles)])

# Define edges (adjacency)
# Binary adjacency matrix
adjacency = np.array([
    [0, 1, 0, 0, 1, 1],  # A connected to B, E, F
    [1, 0, 1, 0, 0, 1],  # B connected to A, C, F
    [0, 1, 0, 1, 0, 0],  # C connected to B, D
    [0, 0, 1, 0, 1, 0],  # D connected to C, E
    [1, 0, 0, 1, 0, 0],  # E connected to A, D
    [1, 1, 0, 0, 0, 0],  # F connected to A, B
])

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Graph visualization
ax1 = axes[0]

# Draw edges first
for i in range(n_nodes):
    for j in range(i + 1, n_nodes):
        if adjacency[i, j] == 1:
            draw_edge(ax1, positions[i, 0], positions[i, 1],
                     positions[j, 0], positions[j, 1],
                     color=COLORS["grid"], width=2)

# Draw nodes
for i, (pos, label) in enumerate(zip(positions, node_labels)):
    draw_node(ax1, pos[0], pos[1], label, COLORS["signal_1"])

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title("Graph G = (V, E)", fontsize=14, fontweight='bold')

# Add annotations
ax1.text(0, -1.4, f"V = {{{', '.join(node_labels)}}}", 
         ha='center', fontsize=10)

# Right: Adjacency matrix
ax2 = axes[1]

im = ax2.imshow(adjacency, cmap='Blues', vmin=0, vmax=1)
ax2.set_xticks(range(n_nodes))
ax2.set_yticks(range(n_nodes))
ax2.set_xticklabels(node_labels)
ax2.set_yticklabels(node_labels)
ax2.set_xlabel("To node")
ax2.set_ylabel("From node")
ax2.set_title("Adjacency Matrix A", fontsize=14, fontweight='bold')

# Add values in cells
for i in range(n_nodes):
    for j in range(n_nodes):
        color = 'white' if adjacency[i, j] == 1 else 'black'
        ax2.text(j, i, str(int(adjacency[i, j])), ha='center', va='center',
                fontsize=11, color=color, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ A graph with 6 nodes and 7 edges, represented as an adjacency matrix")
print(f"  - Nodes (V): {node_labels}")
print(f"  - Number of edges: {int(adjacency.sum() / 2)} (matrix symmetric, divide by 2)")

In [ ]:
# Visualization 2: Four types of graphs

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Small 4-node examples
n_small = 4
small_labels = ["1", "2", "3", "4"]
small_angles = np.linspace(0, 2 * np.pi, n_small, endpoint=False) - np.pi / 2
small_pos = np.column_stack([0.8 * np.cos(small_angles), 0.8 * np.sin(small_angles)])

# Define four graph types
graphs = {
    "Binary Undirected": {
        "matrix": np.array([[0, 1, 0, 1],
                            [1, 0, 1, 0],
                            [0, 1, 0, 1],
                            [1, 0, 1, 0]]),
        "directed": False,
        "description": "Edges: present (1) or absent (0)\nSymmetric matrix"
    },
    "Binary Directed": {
        "matrix": np.array([[0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1],
                            [1, 0, 0, 0]]),
        "directed": True,
        "description": "Edges have direction\nMay be asymmetric"
    },
    "Weighted Undirected": {
        "matrix": np.array([[0.0, 0.8, 0.0, 0.3],
                            [0.8, 0.0, 0.5, 0.0],
                            [0.0, 0.5, 0.0, 0.9],
                            [0.3, 0.0, 0.9, 0.0]]),
        "directed": False,
        "description": "Edges have continuous weights\nSymmetric matrix"
    },
    "Weighted Directed": {
        "matrix": np.array([[0.0, 0.7, 0.0, 0.0],
                            [0.2, 0.0, 0.6, 0.0],
                            [0.0, 0.3, 0.0, 0.8],
                            [0.5, 0.0, 0.4, 0.0]]),
        "directed": True,
        "description": "Weights + direction\nAsymmetric possible"
    }
}

for idx, (title, info) in enumerate(graphs.items()):
    ax = axes.flat[idx]
    matrix = info["matrix"]
    directed = info["directed"]
    
    # Draw edges
    for i in range(n_small):
        for j in range(n_small):
            if matrix[i, j] > 0:
                # Skip duplicates for undirected
                if not directed and j <= i:
                    continue
                    
                weight = matrix[i, j]
                # Line width proportional to weight for weighted graphs
                lw = 3 * weight if "Weighted" in title else 2.5
                
                if directed:
                    # Draw arrow for directed
                    dx = small_pos[j, 0] - small_pos[i, 0]
                    dy = small_pos[j, 1] - small_pos[i, 1]
                    # Shorten arrow to not overlap with node
                    length = np.sqrt(dx**2 + dy**2)
                    scale = (length - 0.25) / length
                    ax.annotate("", 
                               xy=(small_pos[i, 0] + dx * scale, small_pos[i, 1] + dy * scale),
                               xytext=(small_pos[i, 0] + dx * 0.15, small_pos[i, 1] + dy * 0.15),
                               arrowprops=dict(arrowstyle="->", color=COLORS["signal_2"],
                                             lw=lw, mutation_scale=15),
                               zorder=1)
                else:
                    draw_edge(ax, small_pos[i, 0], small_pos[i, 1],
                             small_pos[j, 0], small_pos[j, 1],
                             color=COLORS["signal_2"], width=lw)
    
    # Draw nodes
    for i, (pos, label) in enumerate(zip(small_pos, small_labels)):
        draw_node(ax, pos[0], pos[1], label, COLORS["signal_1"], size=0.12)
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=13, fontweight='bold', pad=10)
    
    # Add description
    ax.text(0, -1.15, info["description"], ha='center', va='top',
           fontsize=9, style='italic', color=COLORS["grid"],)

plt.suptitle("Types of Graphs", fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("✓ Four graph types:")
print("  - Binary: edges are 0 or 1")
print("  - Weighted: edges have continuous strengths")
print("  - Undirected: A is symmetric (PLV, coherence, correlation)")
print("  - Directed: A may be asymmetric (transfer entropy)")

---
## Section 3: From Connectivity Matrix to Graph

Connectivity matrices are **weighted adjacency matrices** — each entry represents the strength of connection between two channels. But many graph metrics require **binary** graphs (edges present or absent).

### Thresholding: Weighted → Binary

To convert a weighted matrix to binary, we **threshold** — keep only connections above a certain value:

$$A_{ij}^{binary} = \begin{cases} 1 & \text{if } |W_{ij}| > \theta \\ 0 & \text{otherwise} \end{cases}$$

### Thresholding Strategies

| Strategy | Description | Use Case |
|----------|-------------|----------|
| **Absolute** | Keep edges above fixed value | When threshold has meaning (e.g., PLV > 0.5) |
| **Proportional** | Keep top X% of edges | Comparing networks with equal density |
| **Significance** | Keep statistically significant edges | When p-values available |

### Trade-offs

- **High threshold** → Sparse graph → May lose important structure
- **Low threshold** → Dense graph → May include noise/spurious connections

**Proportional thresholding** is particularly useful for group comparisons — it ensures all networks have the same **density** (proportion of possible edges that exist).

$$\text{Density} = \frac{\text{actual edges}}{\text{possible edges}} = \frac{\sum_{i \neq j} A_{ij}}{n(n-1)}$$

In [ ]:
# Functions for thresholding connectivity matrices

def threshold_matrix_absolute(
    matrix: NDArray[np.float64],
    threshold: float
) -> NDArray[np.float64]:
    """
    Convert weighted matrix to binary using absolute threshold.
    
    Parameters
    ----------
    matrix : NDArray[np.float64]
        Weighted connectivity matrix.
    threshold : float
        Minimum absolute value to keep edge.
        
    Returns
    -------
    NDArray[np.float64]
        Binary adjacency matrix.
    """
    binary = (np.abs(matrix) > threshold).astype(np.float64)
    np.fill_diagonal(binary, 0)  # No self-connections
    return binary


def threshold_matrix_proportional(
    matrix: NDArray[np.float64],
    density: float
) -> NDArray[np.float64]:
    """
    Convert weighted matrix to binary keeping top proportion of edges.
    
    Parameters
    ----------
    matrix : NDArray[np.float64]
        Weighted connectivity matrix.
    density : float
        Proportion of edges to keep (0-1).
        
    Returns
    -------
    NDArray[np.float64]
        Binary adjacency matrix with specified density.
    """
    n = matrix.shape[0]
    # Get upper triangle values (excluding diagonal)
    triu_indices = np.triu_indices(n, k=1)
    values = np.abs(matrix[triu_indices])
    
    # Find threshold for desired density
    n_edges_to_keep = int(np.ceil(density * len(values)))
    if n_edges_to_keep == 0:
        return np.zeros_like(matrix)
    
    sorted_values = np.sort(values)[::-1]  # Descending
    threshold = sorted_values[min(n_edges_to_keep - 1, len(sorted_values) - 1)]
    
    # Apply threshold
    binary = (np.abs(matrix) >= threshold).astype(np.float64)
    np.fill_diagonal(binary, 0)
    return binary


def get_graph_density(adjacency: NDArray[np.float64]) -> float:
    """
    Compute graph density (proportion of possible edges that exist).
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    float
        Density in range [0, 1].
    """
    n = adjacency.shape[0]
    n_possible = n * (n - 1)  # Exclude diagonal
    n_actual = np.sum(adjacency > 0) - np.trace(adjacency > 0)  # Exclude diagonal
    return n_actual / n_possible if n_possible > 0 else 0.0


print("✓ Thresholding functions defined:")
print("  - threshold_matrix_absolute(): fixed threshold")
print("  - threshold_matrix_proportional(): keep top X% of edges")
print("  - get_graph_density(): compute proportion of edges")

In [ ]:
# Visualization 3: Thresholding process

# Create a simulated weighted connectivity matrix
np.random.seed(42)
n_channels = 8
channel_labels = [f"Ch{i+1}" for i in range(n_channels)]

# Generate random symmetric connectivity matrix
weighted_matrix = np.random.rand(n_channels, n_channels) * 0.6 + 0.2
weighted_matrix = (weighted_matrix + weighted_matrix.T) / 2  # Symmetrize
np.fill_diagonal(weighted_matrix, 0)

# Apply threshold
threshold_value = 0.5
binary_matrix = threshold_matrix_absolute(weighted_matrix, threshold_value)

# Create figure
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Panel 1: Weighted matrix
ax1 = axes[0]
im1 = ax1.imshow(weighted_matrix, cmap='Blues', vmin=0, vmax=1)
ax1.set_xticks(range(n_channels))
ax1.set_yticks(range(n_channels))
ax1.set_xticklabels(channel_labels, fontsize=8)
ax1.set_yticklabels(channel_labels, fontsize=8)
ax1.set_title("Weighted Connectivity\nMatrix", fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=ax1, shrink=0.8, label="Connectivity")

# Panel 2: Binary matrix
ax2 = axes[1]
im2 = ax2.imshow(binary_matrix, cmap='Blues', vmin=0, vmax=1)
ax2.set_xticks(range(n_channels))
ax2.set_yticks(range(n_channels))
ax2.set_xticklabels(channel_labels, fontsize=8)
ax2.set_yticklabels(channel_labels, fontsize=8)
ax2.set_title(f"Binary (threshold = {threshold_value})", fontsize=12, fontweight='bold')

# Add arrow between panels
fig.text(0.37, 0.5, "→", fontsize=30, ha='center', va='center', 
         color=COLORS["signal_4"], fontweight='bold')
fig.text(0.37, 0.35, f"threshold\n> {threshold_value}", fontsize=10, ha='center', va='top',
         color=COLORS["text"])

# Panel 3: Graph visualization
ax3 = axes[2]
angles_8 = np.linspace(0, 2 * np.pi, n_channels, endpoint=False) - np.pi / 2
pos_8 = np.column_stack([np.cos(angles_8), np.sin(angles_8)])

# Draw edges
for i in range(n_channels):
    for j in range(i + 1, n_channels):
        if binary_matrix[i, j] == 1:
            draw_edge(ax3, pos_8[i, 0], pos_8[i, 1],
                     pos_8[j, 0], pos_8[j, 1],
                     color=COLORS["signal_2"], width=1.5)

# Draw nodes
for i, (pos, label) in enumerate(zip(pos_8, channel_labels)):
    draw_node(ax3, pos[0], pos[1], str(i+1), COLORS["signal_1"], size=0.12)

ax3.set_xlim(-1.4, 1.4)
ax3.set_ylim(-1.4, 1.4)
ax3.set_aspect('equal')
ax3.axis('off')
ax3.set_title("Binary Graph", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

density = get_graph_density(binary_matrix)
print(f"✓ Thresholding at {threshold_value}:")
print(f"  - Original: weighted matrix with values in [{weighted_matrix.min():.2f}, {weighted_matrix.max():.2f}]")
print(f"  - Result: binary graph with {int(binary_matrix.sum()/2)} edges")
print(f"  - Graph density: {density:.2%}")

In [ ]:
# Visualization 4: Effect of threshold on network density

thresholds = [0.3, 0.5, 0.7]

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, thresh in zip(axes, thresholds):
    # Apply threshold
    binary = threshold_matrix_absolute(weighted_matrix, thresh)
    density = get_graph_density(binary)
    n_edges = int(binary.sum() / 2)
    
    # Draw graph
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            if binary[i, j] == 1:
                draw_edge(ax, pos_8[i, 0], pos_8[i, 1],
                         pos_8[j, 0], pos_8[j, 1],
                         color=COLORS["signal_2"], width=2)
    
    for i, pos in enumerate(pos_8):
        draw_node(ax, pos[0], pos[1], str(i+1), COLORS["signal_1"], size=0.12)
    
    ax.set_xlim(-1.4, 1.4)
    ax.set_ylim(-1.4, 1.4)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"Threshold = {thresh}\n{n_edges} edges, density = {density:.1%}", 
                fontsize=11, fontweight='bold')

plt.suptitle("Effect of Threshold on Network Density", fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("✓ Threshold affects network structure:")
print("  - Low threshold (0.3): Dense network, may include noise")
print("  - Medium threshold (0.5): Balanced")
print("  - High threshold (0.7): Sparse network, only strongest connections")

---
## Section 4: Node Metrics — Degree and Strength

Now that we can create graphs from connectivity matrices, how do we characterize **individual nodes**? Which brain regions are most connected?

### Degree (Binary Graphs)

The **degree** $k_i$ of a node is the number of edges connected to it:

$$k_i = \sum_{j} A_{ij}$$

- High degree = many connections = potential **hub**
- Low degree = peripheral node

### Strength (Weighted Graphs)

The **strength** $s_i$ of a node is the sum of edge weights:

$$s_i = \sum_{j} W_{ij}$$

- High strength = strong total connectivity
- More informative than degree for weighted networks

### Directed Graphs: In-Degree and Out-Degree

For directed graphs (like transfer entropy networks):

- **In-degree**: edges coming IN → $k_i^{in} = \sum_j A_{ji}$
- **Out-degree**: edges going OUT → $k_i^{out} = \sum_j A_{ij}$

A node with high out-degree is an **information broadcaster** (influences many others).
A node with high in-degree is an **information receiver** (influenced by many others).

In [ ]:
# Functions for node metrics

def compute_degree(adjacency: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Compute degree of each node in a binary graph.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    NDArray[np.float64]
        Degree of each node.
    """
    return np.sum(adjacency > 0, axis=1).astype(np.float64)


def compute_strength(weighted_matrix: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Compute strength of each node in a weighted graph.
    
    Parameters
    ----------
    weighted_matrix : NDArray[np.float64]
        Weighted adjacency matrix.
        
    Returns
    -------
    NDArray[np.float64]
        Strength of each node (sum of edge weights).
    """
    return np.sum(np.abs(weighted_matrix), axis=1)


def compute_in_out_degree(
    adjacency: NDArray[np.float64]
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Compute in-degree and out-degree for a directed graph.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Directed adjacency matrix (A[i,j] = edge from i to j).
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        (in_degree, out_degree) arrays.
    """
    out_degree = np.sum(adjacency > 0, axis=1).astype(np.float64)  # Row sum
    in_degree = np.sum(adjacency > 0, axis=0).astype(np.float64)   # Column sum
    return in_degree, out_degree


print("✓ Node metric functions defined:")
print("  - compute_degree(): number of edges per node")
print("  - compute_strength(): sum of edge weights per node")
print("  - compute_in_out_degree(): for directed graphs")

In [ ]:
# Visualization 5: Node degree and strength

# Use a network with clear hub structure
np.random.seed(123)
n_net = 8

# Create a network with one hub (node 0 connected to all others)
hub_matrix = np.zeros((n_net, n_net))
# Hub node (0) connects to everyone
for i in range(1, n_net):
    strength = np.random.uniform(0.5, 0.9)
    hub_matrix[0, i] = strength
    hub_matrix[i, 0] = strength

# Add some random connections between non-hub nodes
for i in range(1, n_net):
    for j in range(i + 1, n_net):
        if np.random.rand() < 0.3:
            strength = np.random.uniform(0.3, 0.6)
            hub_matrix[i, j] = strength
            hub_matrix[j, i] = strength

# Compute metrics
binary_hub = (hub_matrix > 0).astype(float)
degrees = compute_degree(binary_hub)
strengths = compute_strength(hub_matrix)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Network with node sizes by degree
ax1 = axes[0]
angles_net = np.linspace(0, 2 * np.pi, n_net, endpoint=False) - np.pi / 2
pos_net = np.column_stack([np.cos(angles_net), np.sin(angles_net)])

# Draw edges with width proportional to weight
for i in range(n_net):
    for j in range(i + 1, n_net):
        if hub_matrix[i, j] > 0:
            lw = 1 + 3 * hub_matrix[i, j]
            alpha = 0.3 + 0.5 * hub_matrix[i, j]
            ax1.plot([pos_net[i, 0], pos_net[j, 0]], 
                    [pos_net[i, 1], pos_net[j, 1]],
                    color=COLORS["signal_2"], lw=lw, alpha=alpha, zorder=1)

# Draw nodes sized by degree
max_degree = degrees.max()
for i, (pos, deg) in enumerate(zip(pos_net, degrees)):
    size = 0.08 + 0.12 * (deg / max_degree)
    color = COLORS["signal_4"] if deg == max_degree else COLORS["signal_1"]
    draw_node(ax1, pos[0], pos[1], str(i+1), color, size=size)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title("Node Size ∝ Degree\n(Hub highlighted in gold)", fontsize=12, fontweight='bold')

# Right: Bar chart of degree and strength
ax2 = axes[1]
x = np.arange(n_net)
width = 0.35

bars1 = ax2.bar(x - width/2, degrees, width, label='Degree', color=COLORS["signal_1"])
bars2 = ax2.bar(x + width/2, strengths, width, label='Strength', color=COLORS["signal_2"])

ax2.set_xlabel('Node')
ax2.set_ylabel('Value')
ax2.set_title('Degree vs Strength per Node', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([str(i+1) for i in range(n_net)])
ax2.legend()

# Highlight hub
ax2.axvspan(-0.5, 0.5, alpha=0.2, color=COLORS["signal_4"], label='Hub')

plt.tight_layout()
plt.show()

print("✓ Node 1 is a HUB:")
print(f"  - Highest degree: {int(degrees[0])} connections (vs avg {degrees[1:].mean():.1f} for others)")
print(f"  - Highest strength: {strengths[0]:.2f} (vs avg {strengths[1:].mean():.2f} for others)")

---
## Section 5: Clustering Coefficient — Local Structure

Degree tells us how many connections a node has, but not how those neighbors relate to **each other**. The **clustering coefficient** answers: *Are a node's neighbors also connected to each other?*

### Local Clustering Coefficient

For node $i$ with degree $k_i$:

$$C_i = \frac{\text{triangles around } i}{\text{possible triangles around } i} = \frac{2 \cdot t_i}{k_i(k_i - 1)}$$

where $t_i$ is the number of edges between neighbors of $i$.

- **$C_i = 1$**: All neighbors connected (clique)
- **$C_i = 0$**: No neighbors connected (star)

### Intuition

- **High clustering**: "Friends know each other" — local cohesion, functional modules
- **Low clustering**: Star-like structure — node bridges separate groups

### Global Clustering

Average across all nodes:

$$C = \frac{1}{n} \sum_i C_i$$

Brain networks typically show **high clustering** — indicating modular organization with local processing units.

In [ ]:
# Functions for clustering coefficient

def compute_clustering_coefficient(
    adjacency: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Compute local clustering coefficient for each node.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    NDArray[np.float64]
        Clustering coefficient for each node.
    """
    n = adjacency.shape[0]
    A = (adjacency > 0).astype(float)
    np.fill_diagonal(A, 0)
    
    clustering = np.zeros(n)
    
    for i in range(n):
        neighbors = np.where(A[i, :] > 0)[0]
        k = len(neighbors)
        
        if k < 2:
            clustering[i] = 0.0
            continue
        
        # Count triangles: edges between neighbors
        triangles = 0
        for ni in range(len(neighbors)):
            for nj in range(ni + 1, len(neighbors)):
                if A[neighbors[ni], neighbors[nj]] > 0:
                    triangles += 1
        
        # Possible triangles = k choose 2
        possible = k * (k - 1) / 2
        clustering[i] = triangles / possible
    
    return clustering


def compute_global_clustering(adjacency: NDArray[np.float64]) -> float:
    """
    Compute global clustering coefficient (average over nodes).
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    float
        Global clustering coefficient.
    """
    local_cc = compute_clustering_coefficient(adjacency)
    # Only average over nodes with degree >= 2
    valid = compute_degree(adjacency) >= 2
    if np.sum(valid) == 0:
        return 0.0
    return np.mean(local_cc[valid])


print("✓ Clustering functions defined:")
print("  - compute_clustering_coefficient(): local clustering per node")
print("  - compute_global_clustering(): average clustering")

In [ ]:
# Visualization 6: High vs low clustering comparison

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Create two contrasting networks
n_demo = 6
angles_demo = np.linspace(0, 2 * np.pi, n_demo, endpoint=False) - np.pi / 2
pos_demo = np.column_stack([0.9 * np.cos(angles_demo), 0.9 * np.sin(angles_demo)])

# High clustering: Fully connected (clique)
high_cluster = np.ones((n_demo, n_demo))
np.fill_diagonal(high_cluster, 0)

# Low clustering: Star network (hub connected to all, no other connections)
low_cluster = np.zeros((n_demo, n_demo))
for i in range(1, n_demo):
    low_cluster[0, i] = 1
    low_cluster[i, 0] = 1

networks = [
    ("High Clustering (Clique)", high_cluster, COLORS["signal_3"]),
    ("Low Clustering (Star)", low_cluster, COLORS["signal_2"])
]

for ax, (title, adj, edge_color) in zip(axes, networks):
    # Draw edges
    for i in range(n_demo):
        for j in range(i + 1, n_demo):
            if adj[i, j] > 0:
                ax.plot([pos_demo[i, 0], pos_demo[j, 0]],
                       [pos_demo[i, 1], pos_demo[j, 1]],
                       color=edge_color, lw=2, zorder=1)
    
    # Draw nodes
    for i, pos in enumerate(pos_demo):
        draw_node(ax, pos[0], pos[1], str(i+1), COLORS["signal_1"], size=0.12)
    
    # Compute and display clustering
    cc = compute_clustering_coefficient(adj)
    global_cc = compute_global_clustering(adj)
    
    ax.set_xlim(-1.4, 1.4)
    ax.set_ylim(-1.4, 1.4)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"{title}\nGlobal C = {global_cc:.2f}", fontsize=12, fontweight='bold')
    
    # Show triangles count
    n_triangles = int(np.sum(adj @ adj * adj) / 6)  # Each triangle counted 6 times
    ax.text(0, -1.25, f"Triangles: {n_triangles}", ha='center', fontsize=10)

plt.suptitle("Clustering Coefficient Comparison", fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("✓ Clustering comparison:")
print("  - Clique: Every neighbor connected → C = 1.0 (maximum)")
print("  - Star: No neighbors connected to each other → C = 0.0 (hub has no triangles)")

---
## Section 6: Path Length and Efficiency

While clustering measures **local** structure, we also need metrics for **global** integration. How efficiently can information travel across the entire network?

### Shortest Path

The **shortest path** $d(i,j)$ between nodes $i$ and $j$ is the minimum number of edges to traverse.

### Characteristic Path Length

Average shortest path across all node pairs:

$$L = \frac{1}{n(n-1)} \sum_{i \neq j} d(i,j)$$

- **Low $L$**: Information travels quickly → integrated network
- **High $L$**: Information travels slowly → segregated network

**Problem**: Disconnected nodes have $d(i,j) = \infty$, making $L = \infty$!

### Global Efficiency

Solution: Use the **harmonic mean** of inverse distances:

$$E = \frac{1}{n(n-1)} \sum_{i \neq j} \frac{1}{d(i,j)}$$

- Handles disconnected nodes naturally ($1/\infty = 0$)
- **High $E$**: Efficient network (short paths)
- **Low $E$**: Inefficient network (long paths or disconnected)

In [ ]:
# Functions for path length and efficiency

def compute_shortest_paths(adjacency: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Compute shortest path lengths between all node pairs using Floyd-Warshall.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    NDArray[np.float64]
        Matrix of shortest path lengths (np.inf for disconnected pairs).
    """
    n = adjacency.shape[0]
    A = (adjacency > 0).astype(float)
    np.fill_diagonal(A, 0)
    
    # Initialize distance matrix
    dist = np.full((n, n), np.inf)
    dist[A > 0] = 1  # Direct connections have distance 1
    np.fill_diagonal(dist, 0)
    
    # Floyd-Warshall algorithm
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i, k] + dist[k, j] < dist[i, j]:
                    dist[i, j] = dist[i, k] + dist[k, j]
    
    return dist


def compute_characteristic_path_length(adjacency: NDArray[np.float64]) -> float:
    """
    Compute characteristic path length (average shortest path).
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    float
        Characteristic path length (np.inf if graph is disconnected).
    """
    dist = compute_shortest_paths(adjacency)
    n = adjacency.shape[0]
    
    # Exclude diagonal
    mask = ~np.eye(n, dtype=bool)
    finite_dist = dist[mask]
    
    if np.any(np.isinf(finite_dist)):
        return np.inf
    
    return np.mean(finite_dist)


def compute_global_efficiency(adjacency: NDArray[np.float64]) -> float:
    """
    Compute global efficiency (harmonic mean of inverse path lengths).
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    float
        Global efficiency in range [0, 1].
    """
    dist = compute_shortest_paths(adjacency)
    n = adjacency.shape[0]
    
    # Inverse distances (0 for infinite distances)
    with np.errstate(divide='ignore'):
        inv_dist = 1.0 / dist
    inv_dist[np.isinf(inv_dist)] = 0
    np.fill_diagonal(inv_dist, 0)
    
    # Average over all pairs
    return np.sum(inv_dist) / (n * (n - 1))


print("✓ Path and efficiency functions defined:")
print("  - compute_shortest_paths(): all pairwise shortest paths")
print("  - compute_characteristic_path_length(): average path length")
print("  - compute_global_efficiency(): harmonic mean of inverse paths")

In [ ]:
# Visualization 7: Shortest path example

# Create a chain-like network with one shortcut
n_path = 8
chain_adj = np.zeros((n_path, n_path))

# Chain: 1-2-3-4-5-6-7-8
for i in range(n_path - 1):
    chain_adj[i, i+1] = 1
    chain_adj[i+1, i] = 1

# Add shortcut: 1-5
chain_adj[0, 4] = 1
chain_adj[4, 0] = 1

# Compute paths
dist_matrix = compute_shortest_paths(chain_adj)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Network with path highlighted
ax1 = axes[0]
# Linear layout
pos_chain = np.column_stack([np.linspace(-1.2, 1.2, n_path), np.zeros(n_path)])
# Add some vertical offset for visual interest
pos_chain[3, 1] = 0.3
pos_chain[4, 1] = 0.3
pos_chain[0, 1] = -0.2
pos_chain[7, 1] = -0.2

# Draw all edges
for i in range(n_path):
    for j in range(i + 1, n_path):
        if chain_adj[i, j] > 0:
            color = COLORS["signal_4"] if (i == 0 and j == 4) else COLORS["grid"]
            lw = 3 if (i == 0 and j == 4) else 2
            ax1.plot([pos_chain[i, 0], pos_chain[j, 0]],
                    [pos_chain[i, 1], pos_chain[j, 1]],
                    color=color, lw=lw, zorder=1)

# Highlight shortest path from 1 to 8
path_1_to_8 = [0, 4, 5, 6, 7]  # Using shortcut
for k in range(len(path_1_to_8) - 1):
    i, j = path_1_to_8[k], path_1_to_8[k + 1]
    ax1.plot([pos_chain[i, 0], pos_chain[j, 0]],
            [pos_chain[i, 1], pos_chain[j, 1]],
            color=COLORS["signal_2"], lw=4, alpha=0.7, zorder=2)

# Draw nodes
for i, pos in enumerate(pos_chain):
    color = COLORS["signal_4"] if i in [0, 7] else COLORS["signal_1"]
    draw_node(ax1, pos[0], pos[1], str(i+1), color, size=0.1)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-0.8, 0.8)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title(f"Shortest Path: 1 → 8\nLength = {int(dist_matrix[0, 7])} edges", 
             fontsize=12, fontweight='bold')
ax1.text(0, -0.6, "Gold edge = shortcut that reduces path length", 
        ha='center', fontsize=10, style='italic')

# Right: Distance matrix heatmap
ax2 = axes[1]
im = ax2.imshow(dist_matrix, cmap='YlOrRd', vmin=0, vmax=n_path-1)
ax2.set_xticks(range(n_path))
ax2.set_yticks(range(n_path))
ax2.set_xticklabels([str(i+1) for i in range(n_path)])
ax2.set_yticklabels([str(i+1) for i in range(n_path)])
ax2.set_xlabel("To node")
ax2.set_ylabel("From node")
ax2.set_title("Shortest Path Matrix", fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax2, label="Path length")

# Add values
for i in range(n_path):
    for j in range(n_path):
        val = dist_matrix[i, j]
        color = 'white' if val > 3 else 'black'
        ax2.text(j, i, str(int(val)), ha='center', va='center', 
                fontsize=9, color=color)

plt.tight_layout()
plt.show()

L = compute_characteristic_path_length(chain_adj)
E = compute_global_efficiency(chain_adj)
print(f"✓ Network metrics:")
print(f"  - Characteristic path length L = {L:.2f}")
print(f"  - Global efficiency E = {E:.2f}")
print(f"  - Without shortcut, path 1→8 would be 7 edges (now only 4)")

---
## Section 7: Small-World Networks

Brain networks are neither completely random nor completely regular — they exhibit a special organization called **small-world** topology.

### The Small-World Sweet Spot

| Network Type | Clustering (C) | Path Length (L) | Properties |
|--------------|----------------|-----------------|------------|
| **Regular lattice** | High | High | Local connections only |
| **Random** | Low | Low | Connections are random |
| **Small-world** | **High** | **Low** | Best of both worlds! |

### Small-Worldness Index

Compare observed network to random networks with same size and density:

$$\sigma = \frac{C / C_{random}}{L / L_{random}}$$

- $\sigma > 1$ indicates small-world topology
- Brain networks typically have $\sigma \approx 2-5$

### Why Small-World?

- **High clustering**: Local, specialized processing (functional modules)
- **Short paths**: Efficient global integration (communication across modules)
- **Optimal** for information processing: segregation + integration

In [ ]:
# Functions for network generation and small-worldness

def generate_random_graph(
    n_nodes: int,
    density: float,
    seed: Optional[int] = None
) -> NDArray[np.float64]:
    """
    Generate a random (Erdős-Rényi) graph.
    
    Parameters
    ----------
    n_nodes : int
        Number of nodes.
    density : float
        Proportion of edges to include (0-1).
    seed : int, optional
        Random seed for reproducibility.
        
    Returns
    -------
    NDArray[np.float64]
        Binary adjacency matrix.
    """
    if seed is not None:
        np.random.seed(seed)
    
    adj = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        for j in range(i + 1, n_nodes):
            if np.random.rand() < density:
                adj[i, j] = 1
                adj[j, i] = 1
    
    return adj


def generate_lattice_graph(
    n_nodes: int,
    k_neighbors: int
) -> NDArray[np.float64]:
    """
    Generate a regular ring lattice (each node connected to k nearest neighbors).
    
    Parameters
    ----------
    n_nodes : int
        Number of nodes.
    k_neighbors : int
        Number of neighbors on each side (total degree = 2k).
        
    Returns
    -------
    NDArray[np.float64]
        Binary adjacency matrix.
    """
    adj = np.zeros((n_nodes, n_nodes))
    
    for i in range(n_nodes):
        for k in range(1, k_neighbors + 1):
            j = (i + k) % n_nodes
            adj[i, j] = 1
            adj[j, i] = 1
    
    return adj


def generate_small_world_graph(
    n_nodes: int,
    k_neighbors: int,
    p_rewire: float,
    seed: Optional[int] = None
) -> NDArray[np.float64]:
    """
    Generate a Watts-Strogatz small-world graph.
    
    Start with a ring lattice and randomly rewire edges with probability p.
    
    Parameters
    ----------
    n_nodes : int
        Number of nodes.
    k_neighbors : int
        Initial neighbors on each side.
    p_rewire : float
        Probability of rewiring each edge.
    seed : int, optional
        Random seed.
        
    Returns
    -------
    NDArray[np.float64]
        Binary adjacency matrix.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Start with lattice
    adj = generate_lattice_graph(n_nodes, k_neighbors)
    
    # Rewire edges
    for i in range(n_nodes):
        for k in range(1, k_neighbors + 1):
            if np.random.rand() < p_rewire:
                j = (i + k) % n_nodes
                
                # Remove edge (i, j)
                adj[i, j] = 0
                adj[j, i] = 0
                
                # Add edge to random node (not self, not existing neighbor)
                candidates = [c for c in range(n_nodes) 
                             if c != i and adj[i, c] == 0]
                if candidates:
                    new_j = np.random.choice(candidates)
                    adj[i, new_j] = 1
                    adj[new_j, i] = 1
    
    return adj


print("✓ Network generation functions defined:")
print("  - generate_random_graph(): Erdős-Rényi random network")
print("  - generate_lattice_graph(): regular ring lattice")
print("  - generate_small_world_graph(): Watts-Strogatz small-world")

In [ ]:
# Visualization 8: Three network types comparison

n_sw = 16
k = 3  # Each node connected to 3 neighbors on each side

# Generate three network types
lattice = generate_lattice_graph(n_sw, k)
random_net = generate_random_graph(n_sw, get_graph_density(lattice), seed=42)
small_world = generate_small_world_graph(n_sw, k, p_rewire=0.2, seed=42)

networks = [
    ("Regular Lattice", lattice, COLORS["signal_1"]),
    ("Small-World (p=0.2)", small_world, COLORS["signal_4"]),
    ("Random", random_net, COLORS["signal_2"])
]

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# Circular positions
angles_sw = np.linspace(0, 2 * np.pi, n_sw, endpoint=False) - np.pi / 2
pos_sw = np.column_stack([np.cos(angles_sw), np.sin(angles_sw)])

for ax, (title, adj, node_color) in zip(axes, networks):
    # Draw edges
    for i in range(n_sw):
        for j in range(i + 1, n_sw):
            if adj[i, j] > 0:
                ax.plot([pos_sw[i, 0], pos_sw[j, 0]],
                       [pos_sw[i, 1], pos_sw[j, 1]],
                       color=COLORS["grid"], lw=1, alpha=0.6, zorder=1)
    
    # Draw nodes
    for i, pos in enumerate(pos_sw):
        circle = plt.Circle(pos, 0.08, color=node_color, ec='white', lw=1, zorder=3)
        ax.add_patch(circle)
    
    # Compute metrics
    C = compute_global_clustering(adj)
    L = compute_characteristic_path_length(adj)
    L_str = f"{L:.2f}" if L < 100 else "∞"
    
    ax.set_xlim(-1.4, 1.4)
    ax.set_ylim(-1.4, 1.4)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"{title}\nC = {C:.2f}, L = {L_str}", fontsize=11, fontweight='bold')

plt.suptitle("Network Types: From Lattice to Random", fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Summary table
print("✓ Network comparison:")
print(f"{'Network':<20} {'Clustering (C)':<18} {'Path Length (L)':<18}")
print("-" * 56)
for title, adj, _ in networks:
    C = compute_global_clustering(adj)
    L = compute_characteristic_path_length(adj)
    L_str = f"{L:.2f}" if L < 100 else "∞"
    print(f"{title:<20} {C:<18.2f} {L_str:<18}")

---
## Section 8: Hub Detection

**Hubs** are highly connected, important nodes that play critical roles in network communication. There are multiple ways to identify them:

### Centrality Measures

| Measure | Question | Interpretation |
|---------|----------|----------------|
| **Degree centrality** | How many connections? | Many direct links |
| **Strength centrality** | How strong are connections? | High total connectivity |
| **Betweenness centrality** | How many paths pass through? | Critical for communication |
| **Eigenvector centrality** | Connected to important nodes? | Influential position |

### Betweenness Centrality

The fraction of all shortest paths that pass through node $i$:

$$B_i = \sum_{s \neq i \neq t} \frac{\sigma_{st}(i)}{\sigma_{st}}$$

where $\sigma_{st}$ is the number of shortest paths from $s$ to $t$, and $\sigma_{st}(i)$ is the number passing through $i$.

- High betweenness = "bridge" node
- Damage to high-betweenness nodes disrupts communication

In [ ]:
# Functions for hub detection

def compute_betweenness_centrality(
    adjacency: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Compute betweenness centrality for each node.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
        
    Returns
    -------
    NDArray[np.float64]
        Betweenness centrality for each node (normalized).
    """
    n = adjacency.shape[0]
    dist = compute_shortest_paths(adjacency)
    betweenness = np.zeros(n)
    
    # For each pair of nodes
    for s in range(n):
        for t in range(n):
            if s == t:
                continue
            if np.isinf(dist[s, t]):
                continue
                
            # Check each intermediate node
            for i in range(n):
                if i == s or i == t:
                    continue
                # Node i is on shortest path s→t if d(s,i) + d(i,t) = d(s,t)
                if dist[s, i] + dist[i, t] == dist[s, t]:
                    # Approximate: assume only one shortest path
                    betweenness[i] += 1
    
    # Normalize
    if n > 2:
        betweenness = betweenness / ((n - 1) * (n - 2))
    
    return betweenness


def identify_hubs(
    adjacency: NDArray[np.float64],
    weighted_matrix: Optional[NDArray[np.float64]] = None,
    threshold_percentile: float = 75
) -> Dict[str, Any]:
    """
    Identify hub nodes based on multiple centrality measures.
    
    Parameters
    ----------
    adjacency : NDArray[np.float64]
        Binary adjacency matrix.
    weighted_matrix : NDArray, optional
        Weighted matrix for strength calculation.
    threshold_percentile : float
        Percentile above which nodes are considered hubs.
        
    Returns
    -------
    dict
        Hub analysis results.
    """
    degree = compute_degree(adjacency)
    betweenness = compute_betweenness_centrality(adjacency)
    
    if weighted_matrix is not None:
        strength = compute_strength(weighted_matrix)
    else:
        strength = degree.copy()
    
    # Normalize each metric to 0-1
    def normalize(x):
        if x.max() == x.min():
            return np.zeros_like(x)
        return (x - x.min()) / (x.max() - x.min())
    
    # Composite score (equal weights)
    composite = (normalize(degree) + normalize(strength) + normalize(betweenness)) / 3
    
    # Identify hubs
    threshold = np.percentile(composite, threshold_percentile)
    hub_mask = composite >= threshold
    
    return {
        "hub_indices": np.where(hub_mask)[0],
        "degree": degree,
        "strength": strength,
        "betweenness": betweenness,
        "composite_score": composite
    }


print("✓ Hub detection functions defined:")
print("  - compute_betweenness_centrality(): paths through each node")
print("  - identify_hubs(): multi-metric hub identification")

In [ ]:
# Visualization 9: Hub identification

# Create network with clear hubs
np.random.seed(456)
n_hub = 12

# Create base random network
hub_network = generate_random_graph(n_hub, 0.3, seed=456)

# Add two super-connected hubs (nodes 0 and 6)
for hub_node in [0, 6]:
    for i in range(n_hub):
        if i != hub_node and np.random.rand() < 0.7:
            hub_network[hub_node, i] = 1
            hub_network[i, hub_node] = 1

# Compute hub metrics
hub_results = identify_hubs(hub_network)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Network with nodes colored by composite score
ax1 = axes[0]
angles_hub = np.linspace(0, 2 * np.pi, n_hub, endpoint=False) - np.pi / 2
pos_hub = np.column_stack([np.cos(angles_hub), np.sin(angles_hub)])

# Draw edges
for i in range(n_hub):
    for j in range(i + 1, n_hub):
        if hub_network[i, j] > 0:
            ax1.plot([pos_hub[i, 0], pos_hub[j, 0]],
                    [pos_hub[i, 1], pos_hub[j, 1]],
                    color=COLORS["grid"], lw=1, alpha=0.5, zorder=1)

# Draw nodes sized and colored by composite score
scores = hub_results["composite_score"]
for i, (pos, score) in enumerate(zip(pos_hub, scores)):
    size = 0.08 + 0.12 * score
    is_hub = i in hub_results["hub_indices"]
    color = COLORS["signal_4"] if is_hub else COLORS["signal_1"]
    alpha = 0.6 + 0.4 * score
    
    circle = plt.Circle(pos, size, color=color, ec='white', lw=2, 
                        alpha=alpha, zorder=3)
    ax1.add_patch(circle)
    ax1.text(pos[0], pos[1], str(i+1), ha='center', va='center',
            fontsize=9, fontweight='bold', color='white', zorder=4)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title("Network (Hubs in Gold)", fontsize=12, fontweight='bold')

# Right: Metrics comparison
ax2 = axes[1]
x = np.arange(n_hub)
width = 0.25

ax2.bar(x - width, hub_results["degree"] / hub_results["degree"].max(), 
        width, label='Degree', color=COLORS["signal_1"], alpha=0.8)
ax2.bar(x, hub_results["betweenness"] / (hub_results["betweenness"].max() + 0.01),
        width, label='Betweenness', color=COLORS["signal_2"], alpha=0.8)
ax2.bar(x + width, hub_results["composite_score"], 
        width, label='Composite', color=COLORS["signal_4"], alpha=0.8)

# Highlight hubs
for hub_idx in hub_results["hub_indices"]:
    ax2.axvspan(hub_idx - 0.4, hub_idx + 0.4, alpha=0.2, color=COLORS["signal_4"])

ax2.set_xlabel('Node')
ax2.set_ylabel('Normalized Score')
ax2.set_title('Hub Metrics per Node', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([str(i+1) for i in range(n_hub)])
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

print(f"✓ Identified hubs (top 25%): nodes {[i+1 for i in hub_results['hub_indices']]}")
print(f"  - These nodes have high degree AND high betweenness")

## 9. Network Visualization

**Effective visualization is crucial for understanding brain networks.** Different layout algorithms emphasize different aspects of network structure.

### Layout Algorithms

| Layout | Description | Best For |
|--------|-------------|----------|
| **Circular** | Nodes in a circle | Comparing matrices, ordered data |
| **Spring** | Force-directed (attractive/repulsive) | Revealing clusters |
| **Spectral** | Based on graph Laplacian | Mathematical structure |
| **Anatomical** | Uses electrode positions | EEG topography |

### Key Principles

1. **Node size** → importance (degree, centrality)
2. **Node color** → group membership or metric value
3. **Edge width** → connection strength
4. **Edge color** → positive/negative or different frequency bands

In [ ]:
# Layout algorithms

def spring_layout(
    adjacency: NDArray[np.floating[Any]],
    n_iterations: int = 50,
    seed: Optional[int] = None
) -> NDArray[np.floating[Any]]:
    """
    Compute spring (force-directed) layout for a graph.
    
    Nodes repel each other while edges act as springs.
    
    Parameters
    ----------
    adjacency : NDArray
        Adjacency matrix (n_nodes, n_nodes).
    n_iterations : int
        Number of force simulation iterations.
    seed : int, optional
        Random seed for reproducibility.
        
    Returns
    -------
    NDArray
        Node positions (n_nodes, 2).
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_nodes = adjacency.shape[0]
    
    # Initialize random positions
    pos = np.random.rand(n_nodes, 2) * 2 - 1
    
    k = 1.0 / np.sqrt(n_nodes)  # Optimal distance
    
    for _ in range(n_iterations):
        # Compute pairwise distances
        diff = pos[:, np.newaxis, :] - pos[np.newaxis, :, :]
        dist = np.linalg.norm(diff, axis=2) + 1e-10
        
        # Repulsive forces (all pairs)
        repulsion = k**2 / dist**2
        repulsion = repulsion[:, :, np.newaxis] * diff / dist[:, :, np.newaxis]
        repulsion = np.sum(repulsion, axis=1)
        
        # Attractive forces (connected pairs only)
        attraction = np.zeros_like(pos)
        for i in range(n_nodes):
            for j in range(n_nodes):
                if adjacency[i, j] > 0 and i != j:
                    d = dist[i, j]
                    attraction[i] -= diff[i, j] * d / k
        
        # Update positions
        displacement = repulsion + attraction * 0.1
        displacement_norm = np.linalg.norm(displacement, axis=1, keepdims=True) + 1e-10
        pos += 0.1 * displacement / displacement_norm
        
        # Keep within bounds
        pos = np.clip(pos, -1, 1)
    
    return pos


def spectral_layout(
    adjacency: NDArray[np.floating[Any]]
) -> NDArray[np.floating[Any]]:
    """
    Compute spectral layout using graph Laplacian eigenvectors.
    
    Parameters
    ----------
    adjacency : NDArray
        Adjacency matrix (n_nodes, n_nodes).
        
    Returns
    -------
    NDArray
        Node positions (n_nodes, 2).
    """
    n_nodes = adjacency.shape[0]
    
    # Make symmetric
    adj_sym = (adjacency + adjacency.T) / 2
    
    # Compute Laplacian
    degree_matrix = np.diag(np.sum(adj_sym, axis=1))
    laplacian = degree_matrix - adj_sym
    
    # Get eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian)
    
    # Use 2nd and 3rd eigenvectors for 2D layout (skip 1st - constant)
    pos = eigenvectors[:, 1:3]
    
    # Normalize to [-1, 1]
    pos = pos / (np.max(np.abs(pos)) + 1e-10)
    
    return pos


print("✓ Layout functions defined:")
print("  - spring_layout(): force-directed positioning")
print("  - spectral_layout(): eigenvector-based positioning")

In [ ]:
# Visualization 10: Comparing layout algorithms

# Create a network with some cluster structure
np.random.seed(789)
n_layout = 12

# Create connectivity matrix with two clusters
layout_matrix = np.zeros((n_layout, n_layout))

# Cluster 1: nodes 0-5
for i in range(6):
    for j in range(i+1, 6):
        if np.random.rand() < 0.6:
            layout_matrix[i, j] = layout_matrix[j, i] = 1

# Cluster 2: nodes 6-11  
for i in range(6, 12):
    for j in range(i+1, 12):
        if np.random.rand() < 0.6:
            layout_matrix[i, j] = layout_matrix[j, i] = 1

# Few inter-cluster connections
layout_matrix[2, 8] = layout_matrix[8, 2] = 1
layout_matrix[4, 9] = layout_matrix[9, 4] = 1

# Compute different layouts
n_angles = np.linspace(0, 2*np.pi, n_layout, endpoint=False) - np.pi/2
circular_pos = np.column_stack([np.cos(n_angles), np.sin(n_angles)])
spring_pos = spring_layout(layout_matrix, n_iterations=100, seed=789)
spectral_pos = spectral_layout(layout_matrix)

layouts = [
    ("Circular Layout", circular_pos),
    ("Spring (Force-Directed)", spring_pos),
    ("Spectral Layout", spectral_pos)
]

fig, axes = plt.subplots(1, 3, figsize=(14, 4.5))

for ax, (title, pos) in zip(axes, layouts):
    # Draw edges
    for i in range(n_layout):
        for j in range(i+1, n_layout):
            if layout_matrix[i, j] > 0:
                # Color by whether intra or inter-cluster
                is_inter = (i < 6 and j >= 6) or (i >= 6 and j < 6)
                color = COLORS["signal_4"] if is_inter else COLORS["grid"]
                lw = 2 if is_inter else 1
                ax.plot([pos[i, 0], pos[j, 0]], [pos[i, 1], pos[j, 1]],
                       color=color, lw=lw, alpha=0.6, zorder=1)
    
    # Draw nodes colored by cluster
    for i, p in enumerate(pos):
        color = COLORS["signal_1"] if i < 6 else COLORS["signal_2"]
        circle = plt.Circle(p, 0.08, color=color, ec='white', lw=2, zorder=3)
        ax.add_patch(circle)
        ax.text(p[0], p[1], str(i+1), ha='center', va='center',
               fontsize=8, fontweight='bold', color='white', zorder=4)
    
    ax.set_xlim(-1.4, 1.4)
    ax.set_ylim(-1.4, 1.4)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=11, fontweight='bold')

plt.suptitle("Same Network, Different Layouts\n(Blue = Cluster 1, Orange = Cluster 2, Gold = Inter-cluster edges)",
             fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

print("✓ Spring layout naturally separates clusters")
print("✓ Spectral layout reveals mathematical structure")
print("✓ Circular is good for comparing with adjacency matrices")

## 10. Graph Metrics for EEG Networks

**EEG connectivity analysis applies graph theory to understand brain network organization.** Each electrode becomes a node, and connectivity metrics become edge weights.

### Typical EEG Graph Analysis Pipeline

```
Raw EEG → Preprocessing → Connectivity (PLV, Coherence, etc.) → Thresholding → Graph Metrics
```

### Common Findings in EEG Graph Studies

| Metric | Typical Brain Network Finding |
|--------|------------------------------|
| **Small-world** | Brain networks are small-world (high C, low L) |
| **Hubs** | Posterior regions often act as hubs |
| **Modularity** | Functional modules map to brain regions |
| **Efficiency** | High efficiency enables fast information transfer |

### Key Considerations

- **Threshold selection** affects all downstream metrics
- **Volume conduction** can create spurious connections (use PLI/wPLI)
- **Frequency bands** may show different network organization

In [ ]:
# Visualization 11: Connected vs Disconnected EEG networks

# Create realistic EEG electrode positions (10-20 system simplified)
eeg_labels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
n_eeg = len(eeg_labels)

# 2D positions (approximate head view from above)
eeg_positions = np.array([
    [-0.3, 0.9],   # Fp1 - frontal left
    [0.3, 0.9],    # Fp2 - frontal right
    [-0.5, 0.5],   # F3
    [0.5, 0.5],    # F4
    [-0.7, 0.0],   # C3 - central left
    [0.7, 0.0],    # C4 - central right
    [-0.5, -0.5],  # P3 - parietal left
    [0.5, -0.5],   # P4 - parietal right
    [-0.3, -0.9],  # O1 - occipital left
    [0.3, -0.9],   # O2 - occipital right
])

# Create simulated PLV connectivity matrix with realistic structure
np.random.seed(111)
eeg_connectivity = np.random.rand(n_eeg, n_eeg) * 0.3 + 0.2

# Add stronger intra-hemispheric connections
for i in range(0, n_eeg, 2):  # Left hemisphere
    for j in range(0, n_eeg, 2):
        if i != j:
            eeg_connectivity[i, j] += 0.25
            
for i in range(1, n_eeg, 2):  # Right hemisphere
    for j in range(1, n_eeg, 2):
        if i != j:
            eeg_connectivity[i, j] += 0.25

# Add strong occipital connections (visual network)
eeg_connectivity[8, 9] = eeg_connectivity[9, 8] = 0.85
eeg_connectivity[6, 8] = eeg_connectivity[8, 6] = 0.75
eeg_connectivity[7, 9] = eeg_connectivity[9, 7] = 0.75

# Add inter-hemispheric connections
eeg_connectivity[0, 1] = eeg_connectivity[1, 0] = 0.6  # Fp1-Fp2
eeg_connectivity[2, 3] = eeg_connectivity[3, 2] = 0.55  # F3-F4

# Symmetrize and remove diagonal
eeg_connectivity = (eeg_connectivity + eeg_connectivity.T) / 2
np.fill_diagonal(eeg_connectivity, 0)


def draw_eeg_network(ax, connectivity, positions, labels, threshold, title_suffix=""):
    """Draw EEG network on head outline."""
    binary = (connectivity > threshold).astype(float)
    degrees = compute_degree(binary)
    
    # Draw head outline
    theta_head = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta_head), np.sin(theta_head), 'k-', lw=2)
    ax.plot([0], [1.05], 'v', color='black', markersize=8)  # Nose
    
    # Draw edges
    for i in range(len(labels)):
        for j in range(i+1, len(labels)):
            if connectivity[i, j] > threshold:
                lw = 1 + 2 * (connectivity[i, j] - threshold) / (1 - threshold)
                ax.plot([positions[i, 0], positions[j, 0]],
                       [positions[i, 1], positions[j, 1]],
                       color=COLORS["signal_1"], lw=lw, alpha=0.7, zorder=1)
    
    # Draw nodes
    for i, (pos, label) in enumerate(zip(positions, labels)):
        size = 0.07 + 0.04 * degrees[i] / max(degrees.max(), 1)
        is_isolated = degrees[i] == 0
        color = COLORS["grid"] if is_isolated else (
            COLORS["signal_4"] if degrees[i] == degrees.max() else COLORS["signal_2"])
        circle = plt.Circle(pos, size, color=color, ec='white', lw=1.5, zorder=3)
        ax.add_patch(circle)
        ax.text(pos[0], pos[1], label, ha='center', va='center',
               fontsize=6, fontweight='bold', color='white', zorder=4)
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.axis('off')
    
    return binary, degrees


# Two thresholds: low (connected) and high (disconnected)
threshold_low = 0.5
threshold_high = 0.7

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# ===== ROW 1: LOW THRESHOLD (CONNECTED) =====
binary_low, degrees_low = draw_eeg_network(
    axes[0, 1], eeg_connectivity, eeg_positions, eeg_labels, threshold_low)

L_low = compute_characteristic_path_length(binary_low)
cc_low = compute_global_clustering(binary_low)
eff_low = compute_global_efficiency(binary_low)
is_connected_low = not np.isinf(L_low)

# Matrix
im1 = axes[0, 0].imshow(eeg_connectivity, cmap='RdYlBu_r', vmin=0, vmax=1)
axes[0, 0].axhline(y=-0.5, color='white', lw=0.5)
axes[0, 0].set_xticks(range(n_eeg))
axes[0, 0].set_yticks(range(n_eeg))
axes[0, 0].set_xticklabels(eeg_labels, fontsize=7)
axes[0, 0].set_yticklabels(eeg_labels, fontsize=7)
axes[0, 0].set_title(f'Threshold = {threshold_low}\n(keep edges > {threshold_low})', fontsize=10, fontweight='bold')

# Network title
status_low = "Connected ✓" if is_connected_low else "Disconnected ✗"
axes[0, 1].set_title(f'Network: {status_low}', fontsize=10, fontweight='bold',
                     color=COLORS["high_sync"] if is_connected_low else COLORS["signal_3"])

# Metrics
metrics = ['Path\nLength', 'Clustering', 'Efficiency']
L_display = L_low if is_connected_low else 0
values_low = [L_display, cc_low, eff_low]
bars_low = axes[0, 2].bar(metrics, values_low, 
                          color=[COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_4"]], 
                          alpha=0.8, edgecolor='white', lw=2)
axes[0, 2].set_ylabel('Value')
axes[0, 2].set_title('Network Metrics', fontsize=10, fontweight='bold')
axes[0, 2].set_ylim(0, 2.5)

labels_low = [f'{L_low:.2f}' if is_connected_low else '∞', f'{cc_low:.2f}', f'{eff_low:.2f}']
for bar, lbl in zip(bars_low, labels_low):
    axes[0, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.08,
                    lbl, ha='center', fontsize=9, fontweight='bold')

# ===== ROW 2: HIGH THRESHOLD (DISCONNECTED) =====
binary_high, degrees_high = draw_eeg_network(
    axes[1, 1], eeg_connectivity, eeg_positions, eeg_labels, threshold_high)

L_high = compute_characteristic_path_length(binary_high)
cc_high = compute_global_clustering(binary_high)
eff_high = compute_global_efficiency(binary_high)
is_connected_high = not np.isinf(L_high)

# Matrix with threshold line
im2 = axes[1, 0].imshow(eeg_connectivity, cmap='RdYlBu_r', vmin=0, vmax=1)
axes[1, 0].set_xticks(range(n_eeg))
axes[1, 0].set_yticks(range(n_eeg))
axes[1, 0].set_xticklabels(eeg_labels, fontsize=7)
axes[1, 0].set_yticklabels(eeg_labels, fontsize=7)
axes[1, 0].set_title(f'Threshold = {threshold_high}\n(keep edges > {threshold_high})', fontsize=10, fontweight='bold')

# Network title
status_high = "Connected ✓" if is_connected_high else "Disconnected ✗"
axes[1, 1].set_title(f'Network: {status_high}', fontsize=10, fontweight='bold',
                     color=COLORS["high_sync"] if is_connected_high else COLORS["negative"])

# Metrics
L_display_high = L_high if is_connected_high else 0
values_high = [L_display_high, cc_high, eff_high]
bars_high = axes[1, 2].bar(metrics, values_high, 
                           color=[COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_4"]], 
                           alpha=0.8, edgecolor='white', lw=2)
axes[1, 2].set_ylabel('Value')
axes[1, 2].set_title('Network Metrics', fontsize=10, fontweight='bold')
axes[1, 2].set_ylim(0, 2.5)

labels_high = [f'{L_high:.2f}' if is_connected_high else '∞', f'{cc_high:.2f}', f'{eff_high:.2f}']
for bar, lbl in zip(bars_high, labels_high):
    axes[1, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.08,
                    lbl, ha='center', fontsize=9, fontweight='bold')

# Add annotation for disconnected case
if not is_connected_high:
    axes[1, 2].annotate('Path Length = ∞\n(no path exists!)', 
                        xy=(0, 0), xytext=(0.5, 1.5),
                        fontsize=9, ha='center', color=COLORS["negative"],
                        bbox=dict(boxstyle='round', facecolor='white', edgecolor=COLORS["negative"]))

plt.suptitle("Effect of Threshold on Network Connectivity", fontsize=13, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("✓ Threshold comparison:")
print(f"\n  LOW threshold ({threshold_low}):")
print(f"    - Network is {'CONNECTED' if is_connected_low else 'DISCONNECTED'}")
print(f"    - Path length: {L_low:.2f}" if is_connected_low else f"    - Path length: ∞")
print(f"    - Global efficiency: {eff_low:.2f}")
print(f"\n  HIGH threshold ({threshold_high}):")
print(f"    - Network is {'CONNECTED' if is_connected_high else 'DISCONNECTED'}")
print(f"    - Path length: {L_high:.2f}" if is_connected_high else f"    - Path length: ∞ (some nodes have no path between them)")
print(f"    - Global efficiency: {eff_high:.2f} (still computable!)")
print(f"\n  → When network is disconnected, use Global Efficiency instead of Path Length!")

## 11. Graphs for Hyperscanning

**In hyperscanning, we extend graph analysis to multi-brain networks.** Instead of analyzing one brain, we consider connectivity between two (or more) brains simultaneously.

### Hyperscanning Graph Structure

```
         Subject 1              Subject 2
    ┌─────────────────┐    ┌─────────────────┐
    │  Intra-brain    │    │  Intra-brain    │
    │  connectivity   │    │  connectivity   │
    │   (within S1)   │    │   (within S2)   │
    └────────┬────────┘    └────────┬────────┘
             │     Inter-brain      │
             └──── connectivity ────┘
                  (between S1-S2)
```

### Types of Connections

| Type | Description | What it measures |
|------|-------------|------------------|
| **Intra-brain** | Connections within one brain | Individual neural organization |
| **Inter-brain** | Connections between brains | Neural synchrony during interaction |

### Key Hyperscanning Metrics

- **Inter-brain connectivity density**: How many inter-brain connections exist
- **Intra/Inter ratio**: Balance between within and between brain connections  
- **Cross-brain hubs**: Electrodes that bridge the two brains

In [ ]:
# Visualization 12: Hyperscanning dual-brain network

# Create two "brains" with 5 electrodes each
n_per_brain = 5
n_total = n_per_brain * 2

# Electrode labels
labels_s1 = ['F3₁', 'C3₁', 'P3₁', 'F4₁', 'C4₁']  # Subject 1
labels_s2 = ['F3₂', 'C3₂', 'P3₂', 'F4₂', 'C4₂']  # Subject 2
all_labels = labels_s1 + labels_s2

# Positions: two heads side by side
pos_s1 = np.array([
    [-2.3, 0.5],   # F3
    [-2.7, 0.0],   # C3
    [-2.3, -0.5],  # P3
    [-1.7, 0.5],   # F4
    [-1.3, 0.0],   # C4
])

pos_s2 = np.array([
    [1.7, 0.5],    # F3
    [1.3, 0.0],    # C3
    [1.7, -0.5],   # P3
    [2.3, 0.5],    # F4
    [2.7, 0.0],    # C4
])

all_positions = np.vstack([pos_s1, pos_s2])

# Create hyperscanning connectivity matrix
np.random.seed(222)
hyper_connectivity = np.zeros((n_total, n_total))

# Intra-brain connections (within each subject) - stronger
for brain_start in [0, n_per_brain]:
    for i in range(brain_start, brain_start + n_per_brain):
        for j in range(i + 1, brain_start + n_per_brain):
            hyper_connectivity[i, j] = np.random.rand() * 0.4 + 0.4  # 0.4-0.8
            hyper_connectivity[j, i] = hyper_connectivity[i, j]

# Inter-brain connections (between subjects) - weaker but present
for i in range(n_per_brain):
    for j in range(n_per_brain, n_total):
        # Homologous electrodes have stronger inter-brain sync
        if i == (j - n_per_brain):  # Same electrode position
            hyper_connectivity[i, j] = np.random.rand() * 0.3 + 0.5  # 0.5-0.8
        else:
            hyper_connectivity[i, j] = np.random.rand() * 0.3 + 0.1  # 0.1-0.4
        hyper_connectivity[j, i] = hyper_connectivity[i, j]

# Threshold
threshold_hyper = 0.5
hyper_binary = (hyper_connectivity > threshold_hyper).astype(float)

# Count connection types
intra_s1 = np.sum(hyper_binary[:n_per_brain, :n_per_brain]) / 2
intra_s2 = np.sum(hyper_binary[n_per_brain:, n_per_brain:]) / 2
inter_brain = np.sum(hyper_binary[:n_per_brain, n_per_brain:])

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Left: Connectivity matrix with blocks highlighted
ax1 = axes[0]
im = ax1.imshow(hyper_connectivity, cmap='RdYlBu_r', vmin=0, vmax=1)

# Draw block boundaries
ax1.axhline(y=n_per_brain - 0.5, color='white', lw=2)
ax1.axvline(x=n_per_brain - 0.5, color='white', lw=2)

# Add block labels
ax1.text(n_per_brain/2 - 0.5, -1.2, 'Subject 1', ha='center', fontsize=9, fontweight='bold')
ax1.text(n_per_brain + n_per_brain/2 - 0.5, -1.2, 'Subject 2', ha='center', fontsize=9, fontweight='bold')

ax1.set_xticks(range(n_total))
ax1.set_yticks(range(n_total))
ax1.set_xticklabels(all_labels, fontsize=7, rotation=45)
ax1.set_yticklabels(all_labels, fontsize=7)
ax1.set_title('Hyperscanning Connectivity Matrix', fontsize=11, fontweight='bold')
plt.colorbar(im, ax=ax1, label='PLV', shrink=0.8)

# Annotate blocks
ax1.annotate('Intra S1', xy=(1.5, 1.5), fontsize=8, color='white', ha='center', fontweight='bold')
ax1.annotate('Intra S2', xy=(6.5, 6.5), fontsize=8, color='white', ha='center', fontweight='bold')
ax1.annotate('Inter-brain', xy=(6.5, 1.5), fontsize=8, color='black', ha='center', fontweight='bold')

# Middle: Dual-brain network visualization
ax2 = axes[1]

# Draw two head outlines
for cx in [-2, 2]:
    theta_head = np.linspace(0, 2*np.pi, 100)
    ax2.plot(cx + 0.9*np.cos(theta_head), 0.9*np.sin(theta_head), 'k-', lw=2)
    ax2.plot([cx], [1.0], 'v', color='black', markersize=8)  # Nose

# Draw edges with different colors for intra vs inter
for i in range(n_total):
    for j in range(i+1, n_total):
        if hyper_connectivity[i, j] > threshold_hyper:
            # Determine edge type
            i_is_s1 = i < n_per_brain
            j_is_s1 = j < n_per_brain
            
            if i_is_s1 == j_is_s1:  # Intra-brain
                color = COLORS["signal_1"] if i_is_s1 else COLORS["signal_2"]
                lw = 2
            else:  # Inter-brain
                color = COLORS["signal_4"]
                lw = 3
            
            ax2.plot([all_positions[i, 0], all_positions[j, 0]],
                    [all_positions[i, 1], all_positions[j, 1]],
                    color=color, lw=lw, alpha=0.7, zorder=1)

# Draw nodes
for i, (pos, label) in enumerate(zip(all_positions, all_labels)):
    color = COLORS["signal_1"] if i < n_per_brain else COLORS["signal_2"]
    circle = plt.Circle(pos, 0.12, color=color, ec='white', lw=2, zorder=3)
    ax2.add_patch(circle)
    ax2.text(pos[0], pos[1], label[:2], ha='center', va='center',
            fontsize=7, fontweight='bold', color='white', zorder=4)

ax2.set_xlim(-3.5, 3.5)
ax2.set_ylim(-1.5, 1.5)
ax2.set_aspect('equal')
ax2.axis('off')
ax2.set_title('Dual-Brain Network\n(Gold = Inter-brain connections)', fontsize=11, fontweight='bold')

# Labels
ax2.text(-2, -1.3, 'Subject 1', ha='center', fontsize=10, fontweight='bold', color=COLORS["signal_1"])
ax2.text(2, -1.3, 'Subject 2', ha='center', fontsize=10, fontweight='bold', color=COLORS["signal_2"])

# Right: Connection type breakdown
ax3 = axes[2]
conn_types = ['Intra S1', 'Intra S2', 'Inter-brain']
conn_counts = [intra_s1, intra_s2, inter_brain]
conn_colors = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_4"]]

bars = ax3.bar(conn_types, conn_counts, color=conn_colors, alpha=0.8, edgecolor='white', lw=2)
ax3.set_ylabel('Number of Connections')
ax3.set_title('Connection Types', fontsize=11, fontweight='bold')

for bar, count in zip(bars, conn_counts):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f'{int(count)}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Hyperscanning network analysis:")
print(f"  - Intra-brain connections (S1): {int(intra_s1)}")
print(f"  - Intra-brain connections (S2): {int(intra_s2)}")
print(f"  - Inter-brain connections: {int(inter_brain)}")
print(f"  - Inter/Total ratio: {inter_brain / (intra_s1 + intra_s2 + inter_brain):.2%}")

## 12. Exercises

Test your understanding of graph theory concepts with these exercises.

In [ ]:
# Exercise 1: Compute graph metrics for this network
# Given the adjacency matrix below, compute:
# 1. Degree of each node
# 2. Global clustering coefficient
# 3. Characteristic path length (if connected)

exercise_matrix = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 0],
    [1, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0],
])

# YOUR CODE HERE
# degrees = ...
# clustering = ...
# path_length = ...

# Uncomment to check your answers:
# print(f"Degrees: {degrees}")
# print(f"Global clustering: {clustering:.3f}")
# print(f"Path length: {path_length:.3f}")

In [ ]:
# Exercise 1 - Solution (run to verify)

degrees = compute_degree(exercise_matrix)
clustering = compute_global_clustering(exercise_matrix)
path_length = compute_characteristic_path_length(exercise_matrix)

print("Exercise 1 - Solution:")
print(f"  Degrees: {degrees.astype(int)}")
print(f"  Global clustering: {clustering:.3f}")
print(f"  Path length: {path_length:.3f}" if not np.isinf(path_length) else "  Path length: ∞ (disconnected)")

In [ ]:
# Exercise 2: Threshold effect
# Given the weighted connectivity matrix below, find the threshold that:
# 1. Keeps exactly 6 edges
# 2. Keeps the network connected

weighted_exercise = np.array([
    [0.0, 0.8, 0.3, 0.2],
    [0.8, 0.0, 0.6, 0.4],
    [0.3, 0.6, 0.0, 0.7],
    [0.2, 0.4, 0.7, 0.0],
])

# YOUR CODE HERE
# Try different thresholds and check the number of edges and connectivity

# thresholds_to_try = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
# for t in thresholds_to_try:
#     binary = (weighted_exercise > t).astype(float)
#     n_edges = np.sum(binary) / 2
#     L = compute_characteristic_path_length(binary)
#     print(f"Threshold {t}: {n_edges:.0f} edges, connected={not np.isinf(L)}")

In [ ]:
# Exercise 2 - Solution

print("Exercise 2 - Solution:")
print("Threshold analysis:")
thresholds_to_try = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
for t in thresholds_to_try:
    binary = (weighted_exercise > t).astype(float)
    n_edges = np.sum(binary) / 2
    L = compute_characteristic_path_length(binary)
    connected = "✓" if not np.isinf(L) else "✗"
    print(f"  Threshold {t}: {n_edges:.0f} edges, connected={connected}")

print("\n→ Threshold 0.3 gives exactly 4 edges and keeps the network connected")

## 13. Summary

### Key Concepts

| Concept | Definition | Application |
|---------|------------|-------------|
| **Graph** | Nodes connected by edges | Represent brain regions and their connections |
| **Adjacency Matrix** | Matrix representation of a graph | Store connectivity values |
| **Degree** | Number of connections per node | Identify highly connected regions |
| **Clustering Coefficient** | Local connectivity density | Measure functional segregation |
| **Path Length** | Average shortest path between nodes | Measure integration efficiency |
| **Small-World** | High clustering + short paths | Brain network organization |
| **Hubs** | Highly central nodes | Key information relay points |

### Key Takeaways

1. **Graphs provide a powerful framework** for analyzing brain connectivity
2. **Thresholding is critical** - it determines which connections are considered
3. **Multiple metrics needed** - no single metric captures all network properties
4. **Small-world networks** balance segregation and integration
5. **For hyperscanning**, distinguish intra-brain and inter-brain connections

### What's Next?

In **E02: Introduction to Hyperscanning**, we will:
- Learn about hyperscanning experimental setups
- Understand inter-brain synchronization
- Apply graph concepts to dual-brain data

## 14. Discussion Questions

1. **Why is threshold selection so important in graph analysis?**
   - What happens if threshold is too low? Too high?
   - How would you choose an appropriate threshold for your data?

2. **When would you prefer Global Efficiency over Path Length?**
   - Think about disconnected networks and real-world brain data

3. **What makes brain networks "small-world"?**
   - Why is this organization beneficial for information processing?

4. **In hyperscanning, what does high inter-brain connectivity indicate?**
   - How might this differ between cooperative vs. competitive tasks?

---

*Notebook completed. Run time: ~65 minutes.*